In [ ]:
# Code to read csv file into Colaboratory:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
from pathlib import Path
from datetime import date
from google.colab import files
import pickle

def get_file(id, file):
  download = drive.CreateFile({'id': id})
  download.GetContentFile(file)

In [ ]:
import spacy
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
get_file('1XBs72Z95TaHaTugk8HbRStcqTvQLwsJs', 'train.csv')
get_file('1e4ZGWjSHVsmRF7s062zo7SEzjRp1sWHV', 'test.csv')

In [ ]:
df_train = pd.read_csv('train.csv')
sample = df_train

In [ ]:
sample

,id,raw_address,POI/street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung
3,3,"toko dita, kertosono",toko dita/
4,4,jl. orde baru,/jl. orde baru
...,...,...,...
299995,299995,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani
299996,299996,"raya cila kko, cilandak timur kel.",/raya cila kko
299997,299997,tanjung gusta jl. yaya 2 no 17,/
299998,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/


In [ ]:
raw_address = 'fre bakery medan, meda tin, sei rampah sei rampah'
poi_street = 'french bakery medan/medan-t tin'

# complete_address = 'kh agus salim, no 134 diva, kepek wonosari'
street = poi_street.split('/')[0]
first = raw_address.find(street[0:2])
end = first
end_sub = 0
while raw_address[end] == street[end_sub]:
  end+=1
  end_sub += 1
  if end_sub > len(street) - 1:
    break
  if raw_address[end] != street[end_sub]:
    break

raw_address.replace(raw_address[first:end], street)

'french bakery medan bakery medan, meda tin, sei rampah sei rampah'

In [ ]:
def replace_sub(main, sub):
  split = 3
  if main.find(sub[0:split]) != -1:
    first = main.find(sub[0:split])
  else:
    return main
  end = first
  end_sub = 0
  while main[end] == sub[end_sub]:
    end+=1
    end_sub += 1
    if end_sub > len(sub) - 1 or end > len(main) - 1:
      break
    if main[end] != sub[end_sub]:
      break
  main = main.replace(main[first:end], sub)
  return main

In [ ]:
raw_address = 'desa lasikin, di samping bandara lasikin'
poi_street = 'bandara lasikin/'
correct = poi_street.split('/')

In [ ]:
def find_index(main, sub):
  if main.find(sub) != -1:
    first = main.find(sub)
    end = first+len(sub)
    if end > len(main)-1:
      return (first, len(main)-1)
    return (first, end)
  else:
    return ()


In [ ]:
find_index(raw_address, correct[0])

(25, 39)

In [ ]:
correct[0].split()

['bandara', 'lasikin']

In [ ]:
sample

,id,raw_address,POI/street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung
3,3,"toko dita, kertosono",toko dita/
4,4,jl. orde baru,/jl. orde baru
...,...,...,...
299995,299995,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani
299996,299996,"raya cila kko, cilandak timur kel.",/raya cila kko
299997,299997,tanjung gusta jl. yaya 2 no 17,/
299998,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/


In [ ]:
def new_address(df):
  raw = df['raw_address']
  ans = df['POI/street'].split('/')
  for a in ans:
    if a != '':
      for answer  in a.split():
        raw = replace_sub(raw, answer)
  return raw

def entity_index(df, target_index):
  address = df['new_address']
  target = df['POI/street'].split('/')
  if target[target_index] != '':
    index = find_index(address, target[target_index])
    return index
  else:
    return ()

In [ ]:
sample['new_address'] = sample.apply(lambda row: new_address(row), axis = 1)

In [ ]:
sample.sample(25)

,id,raw_address,POI/street,new_address
18786,18786,kamp jawa mal,/kamp jawa mal,kamp jawa mal
179864,179864,"gandaria utara dwij ii, 6 rt 4 15 kebayoran baru",/dwij ii,"gandaria utara dwij ii, 6 rt 4 15 kebayoran baru"
289158,289158,"toko aneka warna, kan pos, kotabumi udik",toko aneka warna/kan pos,"toko aneka warna, kan pos, kotabumi udik"
39101,39101,"icha salon, sei menc medan krio sunggal",icha salon/sei menc,"icha salon, sei menc medan krio sunggal"
219187,219187,"kios khalisa, peri kem",kios khalisa/peri kem,"kios khalisa, peri kem"
158162,158162,darma kediri binong,darma/,darma kediri binong
148650,148650,"pasir endah dua saudara, sim asih, 121 40199",dua saudara/sim asih,"pasihr endah dua saudara, sim asihh, 121 40199"
192069,192069,ahmad yani no 10b sekar sari ngawi,/,ahmad yani no 10b sekar sari ngawi
138452,138452,buni 37 9 matraman,/buni,buni 37 9 matraman
48130,48130,"kebayoran lama utara biru, sul iska muda no 10...",biru/sul iska muda,"kebayoran lama utara biru, sul iska muda no 10..."


In [ ]:
sample['poi_index'] = sample.apply(lambda row: entity_index(row, 0), axis=1)

In [ ]:
sample['street_index'] = sample.apply(lambda row: entity_index(row, 1), axis=1)

In [ ]:
sample.sample(50)

,id,raw_address,POI/street,new_address,poi_index,street_index
115674,115674,"soejakso. drh msc, kar, banguntapan",soejakso. drh msc/kar,"soejakso. drh msc, kar, banguntapan","(0, 17)","(19, 22)"
218566,218566,jeruk i taman,/jeruk i,jeruk i taman,(),"(0, 7)"
141224,141224,tegal gundil ach adn 43 bogor utara - kota,/ach adn,tegal gundil ach adn 43 bogor utara - kota,(),"(13, 20)"
86167,86167,"pt bumi gasi pers, letj s. par, 70 rw 3 slipi",/letj s. par,"pt bumi gasi pers, letj s. par, 70 rw 3 slipi",(),"(19, 30)"
10276,10276,"ahmad yani, no 1 metro timur",/ahmad yani,"ahmad yani, no 1 metro timur",(),"(0, 10)"
2813,2813,"jatirahayu pat jaya vi, 106 17414 pondok melati",/pat jaya vi,"jatirahayu pat jaya vi, 106 17414 pondok melati",(),"(11, 22)"
24706,24706,"wol mongin salon riris, rw 2",salon riris/wol mongin,"wol mongin salon riris, rw 2","(11, 22)","(0, 10)"
9176,9176,"cv.adi citra, ahmad yani 9",cv.adi citra/ahmad yani,"cv.adi citra, ahmad yani 9","(0, 12)","(14, 24)"
200619,200619,"apo manda, sandu, 36 sandubaya (sandujaya)",apotek mandalika/sandu,"apotek mandalika, sandu, 36 sandubaya (sandujaya)","(0, 16)","(18, 23)"
201703,201703,gg. tape no 78 helvetia timur medan helvetia,/gg. tape,gg. tape no 78 helvetia timur medan helvetia,(),"(0, 8)"


In [ ]:
# delete overlapped index
# if poi_index == street_index and poi_index != ()
# delete that row
# '(4, 19, 'POI')' and '(0, 14, 'STREET')' this is overlap

def overlap(df):
    if df['poi_index'] != () and df['street_index'] != ():
        poi = set([x for x in range(df['poi_index'][0], df['poi_index'][1]+1)])
        street = set([x for x in range(df['street_index'][0], df['street_index'][1]+1)])
        if poi.intersection(street):
            return True
        if street.intersection(poi):
            return True
        return False
    return False

In [ ]:
poi = [x for x in range(4, 19+1)]
street = [x for x in range(0, 14+1)]
print(poi, street)

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


In [ ]:
# delete broken new_address
def broken(df):
    threshold = 20
    if len(df['new_address']) - len(df['raw_address']) > threshold:
           return True
    return False

In [ ]:
test_broken = sample.sample(10)
test_broken['is_broken']= test_broken.apply(lambda row: broken(row), axis=1)
test_broken

,id,raw_address,POI/street,new_address,poi_index,street_index,is_broken
4437,4437,sudi sukoharjo margorejo,/sudi,sudi sukoharjo margorejo,(),"(0, 4)",False
110144,110144,rungkut menanggal rung menan ii d 28 rw 2 gunu...,/rung menan ii d,rungkut menanggal rung menan ii d 28 rw 2 gunu...,(),"(18, 33)",False
283032,283032,man iii kebun bunga banjarmasin timur,/man iii,man iii kebun bunga banjarmasin timur,(),"(0, 7)",False
82025,82025,cika xvi antapani tengah antapani (cicadas),/cika xvi,cika xvi antapani tengah antapani (cicadas),(),"(0, 8)",False
78441,78441,"tung jaya gito gati,",tunggal jaya/gito gati,"tunggal jaya gito gati,","(0, 12)","(13, 22)",False
270941,270941,asian high 2 no 159 kalibanteng kulon 3 semara...,/asian high 2,asian high 2 no 159 kalibanteng kulon 3 semara...,(),"(0, 12)",False
139777,139777,"jl. raya pasar cip re 05, 622 rt 1 cipeundeuy ...",/jl. raya pasar cip re 05,"jl. raya pasar cip re 05, 622 rt 1 cipeundeuy ...",(),"(0, 24)",False
176808,176808,"smp nurudin tonjong, karangjongkeng",smp nurudin tonjong/,"smp nurudin tonjong, karangjongkeng","(0, 19)",(),False
159384,159384,pamoy no 3 pamoyanan rt 4 7 bogor selatan - kota,/pamoy,pamoy no 3 pamoyanan rt 4 7 bogor selatan - kota,(),"(0, 5)",False
175309,175309,"red crystal, danau sun sela, no 36 rw 11",/,"red crystal, danau sun sela, no 36 rw 11",(),(),False


In [ ]:
sample['is_overlap'] = sample.apply(lambda row: overlap(row), axis=1)
sample = sample[~sample['is_overlap']]
sample[~sample['is_overlap']]

,id,raw_address,POI/street,new_address,poi_index,street_index,is_overlap
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika,jl kapuk timur delta sili iii lippo cika 11 a ...,(),"(0, 40)",False
1,1,"aye, jati sampurna",/,"aye, jati sampurna",(),(),False
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung,setu siung 119 rt 5 1 13880 cipayung,(),"(5, 10)",False
3,3,"toko dita, kertosono",toko dita/,"toko dita, kertosono","(0, 9)",(),False
4,4,jl. orde baru,/jl. orde baru,jl. orde baru,(),"(0, 12)",False
...,...,...,...,...,...,...,...
299995,299995,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani,jend ahmad yani 331 kertasari ciamis,(),"(0, 15)",False
299996,299996,"raya cila kko, cilandak timur kel.",/raya cila kko,"raya cila kko, cilandak timur kel.",(),"(0, 13)",False
299997,299997,tanjung gusta jl. yaya 2 no 17,/,tanjung gusta jl. yaya 2 no 17,(),(),False
299998,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/,jalan cipadu jaya taman asri gang bijaksana 3 ...,"(18, 28)",(),False


In [ ]:
sample['is_broken'] = sample.apply(lambda row: broken(row), axis=1)
sample = sample[~sample['is_broken']]
sample[~sample['is_broken']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,raw_address,POI/street,new_address,poi_index,street_index,is_overlap,is_broken
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika,jl kapuk timur delta sili iii lippo cika 11 a ...,(),"(0, 40)",False,False
1,1,"aye, jati sampurna",/,"aye, jati sampurna",(),(),False,False
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung,setu siung 119 rt 5 1 13880 cipayung,(),"(5, 10)",False,False
3,3,"toko dita, kertosono",toko dita/,"toko dita, kertosono","(0, 9)",(),False,False
4,4,jl. orde baru,/jl. orde baru,jl. orde baru,(),"(0, 12)",False,False
...,...,...,...,...,...,...,...,...
299995,299995,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani,jend ahmad yani 331 kertasari ciamis,(),"(0, 15)",False,False
299996,299996,"raya cila kko, cilandak timur kel.",/raya cila kko,"raya cila kko, cilandak timur kel.",(),"(0, 13)",False,False
299997,299997,tanjung gusta jl. yaya 2 no 17,/,tanjung gusta jl. yaya 2 no 17,(),(),False,False
299998,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/,jalan cipadu jaya taman asri gang bijaksana 3 ...,"(18, 28)",(),False,False


In [ ]:
sample.loc[244613]

id                                                         244613
raw_address     menara bank mega semarang. (lantai 3 bag recov...
POI/street             menara bank mega semarang/jalan pandanaran
new_address     menara bank mega semarang. (lantai 3 bag recov...
poi_index                                                 (0, 25)
street_index                                             (51, 67)
is_overlap                                                  False
is_broken                                                   False
Name: 244613, dtype: object

In [ ]:
sample

,id,raw_address,POI/street,new_address,poi_index,street_index,is_overlap,is_broken
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika,jl kapuk timur delta sili iii lippo cika 11 a ...,(),"(0, 40)",False,False
1,1,"aye, jati sampurna",/,"aye, jati sampurna",(),(),False,False
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung,setu siung 119 rt 5 1 13880 cipayung,(),"(5, 10)",False,False
3,3,"toko dita, kertosono",toko dita/,"toko dita, kertosono","(0, 9)",(),False,False
4,4,jl. orde baru,/jl. orde baru,jl. orde baru,(),"(0, 12)",False,False
...,...,...,...,...,...,...,...,...
299995,299995,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani,jend ahmad yani 331 kertasari ciamis,(),"(0, 15)",False,False
299996,299996,"raya cila kko, cilandak timur kel.",/raya cila kko,"raya cila kko, cilandak timur kel.",(),"(0, 13)",False,False
299997,299997,tanjung gusta jl. yaya 2 no 17,/,tanjung gusta jl. yaya 2 no 17,(),(),False,False
299998,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/,jalan cipadu jaya taman asri gang bijaksana 3 ...,"(18, 28)",(),False,False


In [ ]:
# Predict BOTH
# TRAIN_DATA =[(<new_address>, {"entities": [(poi_index, "POI"), (street_index, "STREET")]})]
TRAIN_DATA = []
for index, row in tqdm(sample.iterrows()):
    new_address = row['new_address']
    entities = []
    if len(row['poi_index']) != 0:
        poi_index = list(row['poi_index']) + ['POI']
        entities.append(tuple(poi_index))
    if len(row['street_index']) != 0:
        street_index = list(row['street_index']) + ['STREET']
        entities.append(tuple(street_index))
    train = new_address, {'entities': entities}
    TRAIN_DATA.append(train)

298991it [00:25, 11927.84it/s]


In [ ]:
LABEL_1 = 'POI'
LABEL_2 = 'STREET'

In [ ]:
import pickle
# Step 2
get_file('1SjiWqh2vmdBT7tXG6NmBgs3Ehwzwzr2L', 'nlp_299174_424124-model.pickle')

model_name = 'nlp_299174_424124-model.pickle'
with open(model_name, 'rb') as f:
    # Step 3
    nlp = pickle.load(f)

  

In [ ]:
spacy.require_gpu()

True

In [ ]:
nlp = spacy.blank('id')
# Add pipeline
nlp.add_pipe(nlp.create_pipe('ner'))
nlp.begin_training()
# select ner
ner = nlp.get_pipe('ner')
# Add the new label to ner
ner.add_label(LABEL_1)
ner.add_label(LABEL_2)
# Resume training
optimizer = nlp.resume_training()

In [ ]:
# Importing requirements
from spacy.util import minibatch, compounding
import random
from tqdm import tqdm

for _ in tqdm(range(25)):
    random.shuffle(TRAIN_DATA)
    batches = minibatch(TRAIN_DATA, size=compounding(1, 16, 1.001))
    losses = {}
    for batch in batches:
        #print(*batch)
        texts, annotations = zip(*batch)
        #print(texts, annotations)
        # nlp.update(texts, annotations, drop=0.5, losses=losses)
        nlp.update(texts, annotations, sgd=optimizer, drop=0.15, losses=losses)
    print(losses)

  0%|          | 0/25 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
for _ in tqdm(range(1)):
    random.shuffle(TRAIN_DATA)
    batches = minibatch(TRAIN_DATA, size=compounding(1, 16, 1.001))
    losses = {}
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(texts, annotations, sgd=optimizer, drop=0.2, losses=losses)
    print(losses)


100%|██████████| 1/1 [10:03<00:00, 603.85s/it]

{'ner': 418082.8168788545}


In [ ]:
int(losses['ner'])

89434

In [ ]:
sample.shape[0]

298991

In [ ]:
today = date.today()
name = 'nlp_{}_{}-model.pickle'.format(sample.shape[0], int(losses['ner']))
with open(name, 'wb') as f:
    pickle.dump(nlp, f)
    files.download(name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
get_file('1PmTuS1LXAHnic3GlXg_dwT_2cdr2dv_l', 'test_londre2.csv')

In [ ]:
df_test = pd.read_csv('test_londre2.csv')
test = df_test.copy()

In [ ]:
test = df_test.iloc[0].raw_address
doc = nlp(test)
doc

s. par 53 sidanegara 4 cilacap tengah

In [ ]:
def predict(row):
    doc = nlp(row['raw_address'])
    street = ''
    poi = ''
    for ent in doc.ents:
        if ent.label_ == 'STREET':
            street = ent.text
        if ent.label_ == 'POI':
            poi = ent.text
    return '{}/{}'.format(poi, street)

In [ ]:
test['POI/street'] = test.apply(lambda row: predict(row), axis=1)

In [ ]:
test = test.set_index('id')
final = test.drop(['raw_address'], axis=1)

final.to_csv('result.csv') 
files.download('result.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>